## Sample Prediction Demo

In [1]:
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from src.config import ModelConfig
from src.dataloader import prepare_datasets

2024-10-04 13:51:32.705881: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-04 13:51:32.729092: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-04 13:51:32.735908: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-04 13:51:32.756836: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-04 13:51:34.135106: W tensorflow/compiler/tf2

Segmentation Models: using `keras` framework.


In [16]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import keras
import tensorflow as tf
import segmentation_models as sm

# set up environment
config = ModelConfig()
sm.set_framework('tf.keras')

# prepare data
preprocess_input = sm.get_preprocessing(config.backbone) if config.backbone else None
df = pd.read_csv(config.split_path)
train_ds, val_ds, test_ds = prepare_datasets(
    df,
    batch_size=1, 
    preprocess_func=preprocess_input, 
    augment_flag=False, 
)

# define model
if config.backbone:
    unet_model = sm.Unet(
        config.backbone, 
        encoder_weights=config.encoder_weights, 
        input_shape=config.img_size + (config.num_channels,), 
        classes=1,
        activation='sigmoid'
    )
else:
    unet_model = Unet(
        input_shape=config.img_size + (config.num_channels,), 
        classes=1, 
        activation='sigmoid'
    ).build()

# compile model
unet_model.compile(
    optimizer=Adam(learning_rate=config.learning_rate),
    loss=sm.losses.binary_focal_jaccard_loss,
    metrics=['accuracy', sm.metrics.precision, sm.metrics.recall, sm.metrics.iou_score],
)

Training set size: 273
Validation set size: 10
Test set size: 10


TypeError: Unable to reconstruct an instance of 'SumOfLosses' because the class is missing a `from_config()` method. Full object config: {'module': 'segmentation_models.base.objects', 'class_name': 'SumOfLosses', 'config': 'binary_focal_loss_plus_jaccard_loss', 'registered_name': 'SumOfLosses'}

In [ ]:
# Function to predict and visualize
def predict_and_visualize(image_path):
    # Load and preprocess the image
    original_img = load_img(image_path, target_size=config.img_size)
    preprocessed_img = preprocess_image(image_path)
    
    # Make prediction
    prediction = model.predict(preprocessed_img)
    prediction = np.squeeze(prediction)
    
    # Visualize
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 5))
    
    ax1.imshow(original_img)
    ax1.set_title('Original Image')
    ax1.axis('off')
    
    ax2.imshow(prediction, cmap='gray')
    ax2.set_title('Predicted Mask')
    ax2.axis('off')
    
    ax3.imshow(original_img)
    ax3.imshow(prediction, alpha=0.4, cmap='jet')
    ax3.set_title('Overlay')
    ax3.axis('off')
    
    plt.tight_layout()
    plt.show()

# Get list of sample images
sample_images = [f for f in os.listdir(config.sample_folder) if f.endswith(('.png', '.jpg', '.jpeg'))]

# Predict on sample images
for image_file in sample_images:
    image_path = os.path.join(config.sample_folder, image_file)
    print(f"Predicting on {image_file}")
    predict_and_visualize(image_path)

# Batch prediction (optional)
def batch_predict(image_paths):
    batch = np.vstack([preprocess_image(path) for path in image_paths])
    return model.predict(batch)

# Example of batch prediction
batch_size = 4
for i in range(0, len(sample_images), batch_size):
    batch_paths = [os.path.join(config.sample_folder, img) for img in sample_images[i:i+batch_size]]
    batch_predictions = batch_predict(batch_paths)
    print(f"Batch {i//batch_size + 1} predictions shape: {batch_predictions.shape}")


In [15]:
import json
import h5py
from tensorflow import keras

def inspect_keras_file(filepath):
    """
    Inspect a .keras file to extract information about custom objects and model structure.
    
    Args:
    filepath (str): Path to the .keras file
    
    Returns:
    None, prints information about the model
    """
    with h5py.File(filepath, mode='r') as f:
        # Check if it's a Keras model file
        if 'keras_version' not in f.attrs:
            print("This doesn't appear to be a valid Keras model file.")
            return

        print(f"Keras version: {f.attrs['keras_version']}")

        # Extract model config
        if '_config' in f['model_weights'].attrs:
            model_config = json.loads(f['model_weights'].attrs['_config'])
            print("\nModel Configuration:")
            print(json.dumps(model_config, indent=2))

        # Look for custom objects
        custom_objects = set()
        def extract_custom_objects(config):
            if isinstance(config, dict):
                class_name = config.get('class_name')
                if class_name:
                    custom_objects.add(class_name)
                for value in config.values():
                    extract_custom_objects(value)
            elif isinstance(config, list):
                for item in config:
                    extract_custom_objects(item)

        extract_custom_objects(model_config)

        print("\nPotential Custom Objects:")
        for obj in custom_objects:
            if obj not in dir(keras.layers):
                print(f"- {obj}")

        # Extract compile config if available
        if 'optimizer_config' in f['model_weights'].attrs:
            compile_config = json.loads(f['model_weights'].attrs['optimizer_config'])
            print("\nCompile Configuration:")
            print(json.dumps(compile_config, indent=2))

# Usage
inspect_keras_file(config.save_model_path)

OSError: Unable to synchronously open file (file signature not found)